In [1]:
%load_ext jupyter_black

import notebooks_path

notebooks_path.include_packages()
import jax
import jax.numpy as jnp
import jax.random as jr
import optax
import equinox as eqx
import functools
import tqdm

from vit import dataloader, util, model, train

In [2]:
batch_size = 8

In [3]:
train_dataloader = dataloader.get_train_dataloader(batch_size)
test_dataloder = dataloader.get_test_dataloader(batch_size)

Files already downloaded and verified


Files already downloaded and verified


In [4]:
image = next(iter(test_dataloder))

In [5]:
jax_image = jnp.array(image[0][0].numpy())
print(jax_image.shape)

2024-01-26 11:11:31.298036: W external/xla/xla/service/gpu/nvptx_compiler.cc:698] The NVIDIA driver's CUDA version is 12.1 which is older than the ptxas CUDA version (12.3.107). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


(3, 32, 32)


In [6]:
patches = util.img_to_patches(jax_image, patch_size=16, flatten_channel=False)

In [7]:
util.plot_patches(patches)

In [8]:
patches = util.img_to_patches(jax_image, patch_size=16, flatten_channel=False)

In [9]:
# Hyperparameters
lr = 0.0001
dropout_rate = 0.1
beta1 = 0.9
beta2 = 0.999
patch_size = 4
num_patches = 64
num_steps = 100000
image_size = (32, 32, 3)
embedding_dim = 512
hidden_dim = 256
num_heads = 8
num_layers = 6
height, width, channels = image_size
num_classes = 10

In [10]:
key = jr.PRNGKey(2003)

model_obj = model.VisionTransformer(
    embedding_dim=embedding_dim,
    channels=channels,
    hidden_dim=hidden_dim,
    num_heads=num_heads,
    num_layers=num_layers,
    dropout_rate=dropout_rate,
    patch_size=patch_size,
    num_patches=num_patches,
    num_classes=num_classes,
    key=key,
)

optimizer = optax.adamw(
    learning_rate=lr,
    b1=beta1,
    b2=beta2,
)

state = optimizer.init(eqx.filter(model_obj, eqx.is_inexact_array))

model_obj, state, losses = train.train(
    model_obj, optimizer, state, train_dataloader, batch_size, num_steps, key=key
)

Step: 0/100000, Loss: 2.3389408588409424.


Step: 1000/100000, Loss: 1.543238639831543.


Step: 2000/100000, Loss: 1.8728426694869995.


Step: 3000/100000, Loss: 2.5119857788085938.


Step: 4000/100000, Loss: 1.6348769664764404.


...

Step: 96000/100000, Loss: 0.5107324123382568.


Step: 97000/100000, Loss: 0.8225865364074707.


Step: 98000/100000, Loss: 0.234283447265625.


Step: 99000/100000, Loss: 0.6071439385414124.


Step: 99999/100000, Loss: 0.4844452440738678.


In [11]:
accuracies = []

for batch in range(len(test_dataloder.dataset) // batch_size):
    images, labels = next(iter(test_dataloder))

    logits = jax.vmap(functools.partial(model_obj, enable_dropout=False))(
        images.numpy(), key=jax.random.split(key, num=batch_size)
    )

    predictions = jnp.argmax(logits, axis=-1)

    accuracy = jnp.mean(predictions == labels.numpy())

    accuracies.append(accuracy)

print(f"Accuracy: {jnp.sum(jnp.array(accuracies)) / len(accuracies) * 100}%")

Accuracy: 81.89000701904297%
